<!--
CSI-6-ARI Week 2 Tutorial
Search and Adversarial Games
-->


<style>
  :root{
    --bg:#0b1320;
    --fg:#eef3fb;
    --muted:#b9c6dc;
    --card:#ffffff;
    --line:#e6eaf2;
    --soft:#f6f8fb;
    --info:#eef6ff;
    --warn:#fff7e6;
    --task:#fff0f3;
    --ok:#eefaf0;
    --infoLine:#cfe5ff;
    --warnLine:#ffe1a6;
    --taskLine:#ffd1dc;
    --okLine:#bfe8c7;
  }

  /* Consistent notebook typography (match Week 1 exactly) */
  .markdown, .markdown p, .markdown li, .markdown div { font-size: 16px; line-height: 1.65; }
  h1 { font-size: 36px; margin: 0 0 12px 0; }
  h2 { font-size: 24px; margin: 22px 0 10px 0; }
  h3 { font-size: 19px; margin: 16px 0 8px 0; }

  .hero{
    padding:20px 22px;
    border-radius:18px;
    background:var(--bg);
    color:var(--fg);
    border:1px solid rgba(255,255,255,0.12);
  }
  .hero .subtitle{margin-top:10px;font-size:18px;font-weight:650;color:var(--muted);}
  .hero .meta{margin-top:10px;font-size:14px;color:var(--muted);}

  .grid{display:grid;grid-template-columns:1fr 1fr;gap:12px;margin-top:14px;}
  .card{
    padding:14px 16px;
    border-radius:16px;
    background:var(--card);
    border:1px solid var(--line);
    box-shadow:0 1px 0 rgba(20,30,50,0.04);
  }
  .card h3{margin:0 0 10px 0;font-size:18px;}

  .box{padding:14px 16px;border-radius:16px;border:1px solid var(--line);background:var(--soft);margin:12px 0;}
  .box.info{background:var(--info);border-color:var(--infoLine);}
  .box.warn{background:var(--warn);border-color:var(--warnLine);}
  .box.task{background:var(--task);border-color:var(--taskLine);}
  .box.ok{background:var(--ok);border-color:var(--okLine);}

  .boxtitle{font-weight:850;font-size:18px;margin:0 0 8px 0;display:flex;gap:10px;align-items:center;}

  .badge{
    width:28px;height:28px;
    border-radius:9px;
    display:inline-flex;align-items:center;justify-content:center;
    font-weight:900;font-size:16px;
    border:1px solid rgba(0,0,0,0.08);
  }
  .b-info{background:#dbeafe;color:#1d4ed8;}
  .b-warn{background:#ffedd5;color:#c2410c;}
  .b-task{background:#ffe4e6;color:#be123c;}
  .b-ok{background:#dcfce7;color:#166534;}

  code{background:#f1f5f9;border-radius:6px;padding:1px 6px;}
  details{border:1px dashed #cbd5e1;border-radius:14px;padding:10px 12px;background:#fbfdff;}
  summary{cursor:pointer;font-weight:850;font-size:16px;}
</style>

<div class="hero">
  <h1><b>CSI-6-ARI, Week 2 Tutorial</b></h1>
  <div class="subtitle">Search and Adversarial Games</div>
</div>

<div class="grid">
  <div class="card">
    <h3><b>🎯 Learning outcomes</b></h3>
    <ul>
      <li>Run BFS and DFS on graph problems, and interpret the effect of neighbour ordering.</li>
      <li>Use heuristic search (Greedy Best-First Search) and explain how <code>h(n)</code> influences exploration.</li>
      <li>Run A* search and interpret <code>f(n)=g(n)+h(n)</code> in terms of cost and guidance.</li>
      <li>Apply adversarial search (Minimax).</li>
    </ul>
  </div>
  <div class="card">
    <h3><b>🧭 How to use this notebook</b></h3>
    <ul>
      <li>Run cells top-to-bottom. Later sections assume earlier variables exist.</li>
      <li>Exercises use an empty code cell. Complete it before opening the answer.</li>
      <li>Answers are in collapsible boxes with explanation.</li>
    </ul>
  </div>
</div>


<div class="box warn">
  <div class="boxtitle"><span class="badge b-warn">⚠️</span> Important</div>
  <ul>
    <li>We keep this notebook intentionally simple, fewer abstractions, fewer helper functions.</li>
    <li>Focus on the algorithmic ideas first, then (optionally) tidy the code later.</li>
  </ul>
</div>


In [1]:
# ✅ Setup
# We fix a random seed so that any random examples (if used) behave the same for everyone.

import math                 # Basic maths functions (e.g., sqrt, log), used in heuristics and calculations
from collections import deque  # Efficient queue for BFS (fast append/pop from both ends)
import heapq                # Priority queue for GBFS / A* (always pops the lowest-priority item)

SEED = 42                   # Constant used to make randomness reproducible (when random numbers are introduced)
print("Setup complete, seed =", SEED)  # Quick confirmation that the environment is ready


Setup complete, seed = 42


## 1) <b>✅ Uninformed search, BFS and DFS</b>

<div class="box">
  <div class="boxtitle"><span class="badge b-info">🧭</span> Core idea</div>
  <ul>
    <li>
      <b>BFS (Breadth-First Search)</b> explores level-by-level using a <b>FIFO queue</b>.
      It expands all nodes at depth 0, then depth 1, then depth 2, etc.
      With equal step costs, BFS finds the <b>shortest path in number of steps</b> (fewest edges).
    </li>
    <li>
      <b>DFS (Depth-First Search)</b> explores deep paths first using a <b>LIFO stack</b> (often implemented via recursion or a Python list).
      It can reach a goal quickly in some cases, but it is <b>not guaranteed</b> to find the shortest path, and it can waste time in deep dead-ends.
    </li>
    <li>
      In <b>graph search</b> (not tree search), we keep an <b>explored set</b> (or <b>visited</b> set) to avoid revisiting states.
      This prevents infinite loops in cyclic graphs and reduces repeated work.
    </li>
    <li>
      <b>Key terms:</b> the <b>frontier</b> is the data structure holding nodes to be expanded next (queue for BFS, stack for DFS),
      and <b>expanding</b> a node means generating its neighbours/successors.
    </li>
  </ul>
</div>



In [2]:
def bfs(graph, start, goal):
    frontier = deque([start])          # FIFO queue of states to expand next (BFS frontier)
    parent = {start: None}             # Parent pointers for path reconstruction
    explored = set([start])            # Visited set to avoid revisiting nodes (prevents cycles)
    expanded = 0                       # Count how many nodes we expand (basic effort measure)

    while frontier:
        s = frontier.popleft()         # Pop from the left, FIFO behaviour
        expanded += 1                  # We are expanding node s now

        if s == goal:                 # Goal test when we remove from the queue
            # Reconstruct path by following parent pointers from goal back to start
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expanded  # Return start→goal path and expansion count

        for nxt in graph.get(s, []):   # Generate neighbours (successor states)
            if nxt not in explored:    # Only add unseen nodes
                explored.add(nxt)      # Mark visited as soon as we enqueue (avoids duplicate enqueues)
                parent[nxt] = s        # Record how we reached nxt
                frontier.append(nxt)   # Enqueue neighbour for later expansion

    return None, expanded              # No path found; return None plus how many nodes were expanded


In [3]:
def dfs(graph, start, goal):
    frontier = [start]                 # LIFO stack of states to expand next (DFS frontier)
    parent = {start: None}             # Parent pointers for path reconstruction
    explored = set([start])            # Visited set to avoid revisiting nodes (prevents cycles)
    expanded = 0                       # Count how many nodes we expand (basic effort measure)

    while frontier:
        s = frontier.pop()             # Pop from the end, LIFO behaviour
        expanded += 1                  # We are expanding node s now

        if s == goal:                  # Goal test when we pop from the stack
            # Reconstruct path by following parent pointers from goal back to start
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expanded  # Return start→goal path and expansion count

        # reversed(...) helps keep neighbour expansion order comparable to BFS when using a stack
        for nxt in reversed(graph.get(s, [])):
            if nxt not in explored:    # Only push unseen nodes
                explored.add(nxt)      # Mark visited as soon as we push (avoids duplicate pushes)
                parent[nxt] = s        # Record how we reached nxt
                frontier.append(nxt)   # Push neighbour onto the stack for deep-first exploration

    return None, expanded              # No path found; return None plus how many nodes were expanded


<div class="box info">
  <div class="boxtitle"><span class="badge b-info">🧪</span> Worked example, BFS vs DFS on a directed graph</div>
  <p>We translate this diagram into an adjacency list, then compare BFS and DFS from <b>A</b> to the goal node <b>G</b>.</p>
</div>

<div style="padding:12px 14px;border-radius:12px;background:#ffffff;border:1px solid var(--line);
            font-family:ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono','Courier New', monospace;
            font-size:15px;line-height:1.5;">
<pre style="margin:0;">
Directed graph (goal node G)

A ──► B ──► D ──► G
│     │
│     └──► E ──► F ──► G
│
└──► C ──► H ──► I ──► G
</pre>
</div>

<div class="box">
  <div class="boxtitle"><span class="badge b-info">🗒️</span> Notes</div>
  <ul>
    <li><b>Directed edges:</b> you can only move along the arrow direction.</li>
    <li><b>Multiple routes to G:</b> there is more than one valid path from <code>A</code> to <code>G</code>.</li>
    <li><b>Equal step cost assumption:</b> treat each arrow as “1 step”. Under this assumption, <b>BFS</b> finds the fewest steps.</li>
    <li><b>DFS depends on neighbour order:</b> DFS may find a longer path first depending on how neighbours are listed in the adjacency list.</li>
    <li><b>Explored set:</b> we track visited nodes so the algorithms do not waste time revisiting the same state.</li>
  </ul>
</div>


In [ ]:
# Step 1: Encode the diagram as an adjacency list (directed graph)
# Each key is a node, and its list value contains the outgoing neighbours (in order).
graph1 = {
    "A": ["B", "C"],   # From A you can go to B or C
    "B": ["D", "E"],   # From B you can go to D or E
    "C": ["H"],        # From C you can go to H
    "D": ["G"],        # From D you can go directly to G
    "E": ["F"],        # From E you can go to F
    "F": ["G"],        # From F you can go to G
    "H": ["I"],        # From H you can go to I
    "I": ["G"],        # From I you can go to G
    "G": []            # G is the goal, no outgoing edges needed here
}

start, goal = "A", "G"  # We search for a path from A (start) to G (goal)

# Run BFS and DFS on the same graph so we can compare their behaviour
bfs_path, bfs_expanded = bfs(graph1, start, goal)  # BFS returns the path found and how many nodes were expanded
dfs_path, dfs_expanded = dfs(graph1, start, goal)  # DFS returns the path found and how many nodes were expanded

# Print results in a compact, readable form
print("BFS path:", bfs_path, "| expanded:", bfs_expanded)
print("DFS path:", dfs_path, "| expanded:", dfs_expanded)


BFS path: ['A', 'B', 'D', 'G'] | expanded: 7
DFS path: ['A', 'B', 'D', 'G'] | expanded: 4


<b>=============================  EXERCISE  =============================</b>


<div class="box task">
  <div class="boxtitle"><span class="badge b-task">✍️</span> <b>Exercise 1, BFS and DFS</b></div>

  <p><b>Goal:</b> practise encoding a graph and comparing how <b>BFS</b> and <b>DFS</b> behave on the same problem.</p>

  <ol>
    <li>
      <b>Build the adjacency list</b> (a Python dictionary) for the dungeon graph below.
      Each key should be a node (e.g., <code>"S"</code>) and the value should be a list of neighbours in the given order.
    </li>
    <li>
      <b>Run BFS and DFS</b> from <b>S</b> (start) to <b>T</b> (target/goal) using the functions provided earlier.
    </li>
    <li>
      <b>Report your results</b>:
      <ul>
        <li>the <b>path</b> returned by BFS and DFS (start → goal)</li>
        <li>the number of <b>expanded nodes</b> for each algorithm</li>
      </ul>
    </li>
  </ol>

  <p><b>Tip:</b> keep neighbour order consistent with the list below. DFS is sensitive to ordering because it uses a stack.</p>
  <p><b>Check:</b> if your graph is encoded correctly, BFS should usually find a short path (fewest edges), while DFS may return a different (sometimes longer) path.</p>
</div>

<div style="padding:12px 14px;border-radius:12px;background:#ffffff;border:1px solid var(--line);
            font-family:ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono','Courier New', monospace;
            font-size:15px;line-height:1.5;">
<pre style="margin:0;">
Graph (directed edges)

S connects to: A, B
A connects to: C, D
B connects to: D, E
C connects to: F
D connects to: F, G
E connects to: G
F connects to: T
G connects to: T
T connects to: (none)
</pre>
</div>


In [ ]:
# TODO: Exercise 1 solution area


<details>
  <summary><b>✅ Show answer (Exercise 1, BFS and DFS)</b></summary>

  <p><b>Step 1, build the adjacency list</b></p>

  <pre><code class="language-python">graph_ex1 = {
    "S": ["A", "B"],
    "A": ["C", "D"],
    "B": ["D", "E"],
    "C": ["F"],
    "D": ["F", "G"],
    "E": ["G"],
    "F": ["T"],
    "G": ["T"],
    "T": []
}</code></pre>

  <p><b>Step 2, run BFS and DFS from S to T</b></p>

  <pre><code class="language-python">start, goal = "S", "T"

bfs_path, bfs_expanded = bfs(graph_ex1, start, goal)
dfs_path, dfs_expanded = dfs(graph_ex1, start, goal)

print("BFS path:", bfs_path, "| expanded:", bfs_expanded)
print("DFS path:", dfs_path, "| expanded:", dfs_expanded)</code></pre>

  <p><b>Step 3, interpretation</b></p>
  <ul>
    <li><b>BFS</b> returns a shortest path in number of steps (fewest edges).</li>
    <li><b>DFS</b> may return a different path depending on neighbour order (stack behaviour).</li>
  </ul>

</details>


<b>===================================================================</b>


## 2) <b>✅ Heuristic search, Greedy Best-First Search (GBFS)</b>

<div class="box">
  <div class="boxtitle"><span class="badge b-info">🧠</span> Core idea</div>
  <ul>
    <li>
      A <b>heuristic</b> <code>h(n)</code> is an estimate of how far a state <code>n</code> is from the goal.
      In pathfinding, it often estimates <b>remaining distance</b> (for example, straight-line distance to the target).
    </li>
    <li>
      <b>GBFS (Greedy Best-First Search)</b> always expands the node with the <b>smallest</b> heuristic value <code>h(n)</code>,
      using a <b>priority queue</b>. It is “greedy” because it focuses on what looks closest to the goal right now.
    </li>
    <li>
      <b>What GBFS ignores:</b> it does <b>not</b> consider how expensive the path has been so far.
      In other words, it ignores the accumulated cost <code>g(n)</code> and uses only <code>h(n)</code>.
    </li>
    <li>
      <b>Why it can be fast:</b> if the heuristic is informative, GBFS quickly moves toward the goal and may expand far fewer nodes than BFS.
    </li>
    <li>
      <b>Why it is not optimal:</b> GBFS can choose a path that <i>looks</i> promising but is actually longer or leads into dead-ends.
      It may find a solution quickly, but not the <b>shortest</b> (or lowest-cost) solution.
    </li>
    <li>
      <b>Practical note:</b> like BFS/DFS graph search, we still keep an <b>explored</b>/<b>visited</b> set to avoid cycles and repeated expansions.
    </li>
  </ul>
</div>



In [4]:
# ✅ Heuristics on a grid
# We store node coordinates so we can compute heuristic distances to the goal.

coords = {
    "S": (0, 0),  # Start node
    "A": (1, 0),
    "B": (2, 0),
    "C": (0, 1),
    "D": (1, 1),
    "E": (2, 1),
    "T": (2, 2),  # Target/goal node
}

# Unweighted directed graph (each move costs 1 step)
# Neighbour order matters for tie-breaking when heuristic values are equal.
graph2 = {
    "S": ["A", "C"],     # From S you can go right to A, or up to C
    "A": ["B", "D"],     # From A you can go to B or D
    "B": ["E"],          # From B you can go to E
    "C": ["D"],          # From C you can go to D
    "D": ["E", "T"],     # From D you can go to E or directly to T
    "E": ["T"],          # From E you can go to T
    "T": []              # Goal node, no outgoing edges
}


def manhattan(a, b):
    """Manhattan distance on a grid: |dx| + |dy| (moves like a rook in chess)."""
    (x1, y1) = coords[a]        # Coordinates of current node a
    (x2, y2) = coords[b]        # Coordinates of goal node b
    return abs(x1 - x2) + abs(y1 - y2)  # Heuristic estimate of remaining steps


def gbfs(graph, start, goal, h):
    """Greedy Best-First Search (GBFS) using heuristic h(n) to choose what to expand next."""
    # Frontier is a priority queue (min-heap) of (heuristic_value, state).
    # GBFS always pops the state that currently looks closest to the goal.
    frontier = [(h(start, goal), start)]

    parent = {start: None}      # Parent pointers for path reconstruction
    explored = set([start])     # Visited set to avoid revisiting nodes (graph search)
    expanded = 0                # Count how many nodes were expanded

    while frontier:
        _, s = heapq.heappop(frontier)  # Pick the node with smallest heuristic h(n)
        expanded += 1                   # We are expanding s now

        if s == goal:                   # Goal test
            # Reconstruct path by following parent pointers back to start
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), expanded

        # Expand neighbours and push them into the frontier with priority = h(neighbour)
        for nxt in graph.get(s, []):
            if nxt not in explored:
                explored.add(nxt)               # Mark visited when enqueued (avoids duplicates)
                parent[nxt] = s                 # Record how we reached nxt
                heapq.heappush(frontier, (h(nxt, goal), nxt))  # Greedy: priority is only h

    return None, expanded              # No path found


# Run GBFS from S to T using Manhattan distance as the heuristic
path_gbfs, exp_gbfs = gbfs(graph2, "S", "T", manhattan)

# Report the path and how much search effort was needed
print("GBFS path:", path_gbfs, "| expanded:", exp_gbfs)



GBFS path: ['S', 'A', 'B', 'E', 'T'] | expanded: 5


<b>=============================  EXERCISE  =============================</b>


<div class="box task">
  <div class="boxtitle"><span class="badge b-task">✍️</span> <b>Exercise 2, Heuristic design</b></div>

  <p><b>Goal:</b> compare Manhattan vs Euclidean heuristics in GBFS.</p>

  <ol>
    <li>
      <b>Implement Euclidean distance</b> as <code>euclidean(a, b)</code> using <code>coords</code>.
      Use: <code>sqrt((x1-x2)**2 + (y1-y2)**2)</code>
    </li>
    <li>
      <b>Run GBFS twice</b> from <code>S</code> to <code>T</code>:
      <ul>
        <li>GBFS + <b>Manhattan</b> heuristic</li>
        <li>GBFS + <b>Euclidean</b> heuristic</li>
      </ul>
      Record the <b>path</b> and <b>expanded nodes</b> for each.
    </li>
    <li>
      <b>Explain (2–3 sentences):</b> do you always get the same path? If not, why (tie-breaking, neighbour order, and GBFS ignoring path cost <code>g(n)</code>)?
    </li>
  </ol>

  <p><b>Tip:</b> keep neighbour order in <code>graph2</code> unchanged so the comparison is fair.</p>
</div>


In [ ]:
# TODO: Exercise 2 solution area


<details>
  <summary><b>Show one possible answer (Exercise 2)</b></summary>

  <p><b>Step 1, implement Euclidean heuristic</b></p>
  <pre><code class="language-python">def euclidean(a, b):
    (x1, y1) = coords[a]
    (x2, y2) = coords[b]
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)</code></pre>

  <p><b>Step 2, run GBFS with Manhattan and Euclidean</b></p>
  <pre><code class="language-python">start, goal = "S", "T"

path_m, exp_m = gbfs(graph2, start, goal, manhattan)
path_e, exp_e = gbfs(graph2, start, goal, euclidean)

print("GBFS (Manhattan) path:", path_m, "| expanded:", exp_m)
print("GBFS (Euclidean) path:", path_e, "| expanded:", exp_e)</code></pre>

  <p><b>Step 3, explanation (why paths/expansions can differ)</b></p>
  <ul>
    <li>GBFS chooses the next node using only <code>h(n)</code>, so different heuristics can change the expansion order.</li>
    <li>If multiple nodes have equal (or very similar) heuristic values, <b>tie-breaking</b> and <b>neighbour order</b> can change the path found.</li>
    <li>Because GBFS ignores the path cost so far (<code>g(n)</code>), it is not guaranteed to return the shortest path.</li>
  </ul>

</details>


<b>===================================================================</b>


## 3) <b>✅ Informed search, A* on a weighted graph</b>

<div class="box">
  <div class="boxtitle"><span class="badge b-info">⭐</span> Core idea</div>
  <ul>
    <li>
      <b>A*</b> expands the node with the smallest
      <b><code>f(n) = g(n) + h(n)</code></b>,
      where <code>f(n)</code> is the estimated total cost of a solution path going through <code>n</code>.
    </li>
    <li>
      <b><code>g(n)</code> (cost so far)</b> is the exact path cost from the start to <code>n</code>.
      On a <b>weighted</b> graph, edges can have different costs, so the “best” path is the one with the <b>lowest total cost</b>, not necessarily the fewest steps.
    </li>
    <li>
      <b><code>h(n)</code> (heuristic)</b> estimates the remaining cost from <code>n</code> to the goal.
      A good heuristic guides the search toward the goal while still allowing cheaper alternatives to win when <code>g(n)</code> is large.
    </li>
    <li>
      If <code>h(n)</code> is <b>admissible</b> (it never overestimates the true remaining cost), then A* is <b>optimal</b>,
      meaning it will return the <b>lowest-cost</b> path to the goal (when a solution exists).
    </li>
    <li>
      <b>Practical note:</b> A* typically uses a <b>priority queue</b> for its frontier. We also track the best known <code>g</code>-cost for each node,
      so we can ignore worse routes and avoid unnecessary expansions.
    </li>
  </ul>
</div>



In [5]:
# ✅ A* implementation (weighted graph)
# Weighted graph representation:
#   graph_w = {"S": [("A", 2), ("B", 5)], "A": [("G", 3)], ...}
# Each edge is a (neighbour, cost) tuple, and costs can differ between edges.


def astar(graph_w, start, goal, h):
    # Frontier is a priority queue (min-heap). Each item is (f, g, state):
    #   g = cost so far from start to this state
    #   f = estimated total cost = g + h(state)
    frontier = [(h(start, goal), 0, start)]  # start has g=0, f=h(start)

    parent = {start: None}   # Parent pointers for reconstructing the final path
    best_g = {start: 0}      # Best known g-cost to each node (for pruning worse routes)
    expanded = 0             # Count how many nodes were expanded

    while frontier:
        f, g, s = heapq.heappop(frontier)  # Pop node with smallest f-value
        expanded += 1                       # We are expanding s now

        if s == goal:                       # Goal test
            # Reconstruct path by following parent pointers back to start
            path = []
            while s is not None:
                path.append(s)
                s = parent[s]
            return list(reversed(path)), g, expanded  # Return path, total cost g, and expansions

        # Expand all outgoing edges (s -> nxt) with given edge cost
        for nxt, cost in graph_w.get(s, []):
            new_g = g + cost                # Candidate cost to reach nxt via s

            # Only keep this route if it is better (lower cost) than any previously found route to nxt
            if nxt not in best_g or new_g < best_g[nxt]:
                best_g[nxt] = new_g
                parent[nxt] = s
                heapq.heappush(frontier, (new_g + h(nxt, goal), new_g, nxt))  # f = g + h

    return None, math.inf, expanded          # No path found


# Worked example: same graph, compare GBFS vs A*
# Note: GBFS ignores edge costs, it only uses h(n) to decide what to expand next.

coords3 = {
    "S": (0, 0),  # Start
    "A": (1, 0),
    "B": (1, 1),
    "C": (0, 1),
    "G": (2, 0)   # Goal
}

def manhattan3(a, b):
    """Manhattan distance heuristic on the coordinate grid: |dx| + |dy|."""
    (x1, y1) = coords3[a]
    (x2, y2) = coords3[b]
    return abs(x1 - x2) + abs(y1 - y2)

graph3 = {
    # From S, all moves cost 1 (same step cost initially)
    "S": [("A", 1), ("B", 1), ("C", 1)],

    # But the final edges to G have very different costs
    # A looks close to G on the grid, but the edge cost is expensive (10)
    "A": [("G", 10)],

    # B and C are slightly "further" by heuristic, but their edges to G are cheap (2)
    "B": [("G", 2)],
    "C": [("G", 2)],

    "G": []
}

# For GBFS we need an unweighted adjacency list (it does not use edge costs)
adj3 = {k: [n for (n, _) in v] for k, v in graph3.items()}

# Compare outputs:
# - GBFS returns a path and expansions, but may choose an expensive path because it ignores costs.
# - A* returns (path, total_cost, expansions) and should choose the cheapest path if h is admissible.
print("GBFS:", gbfs(adj3, "S", "G", manhattan3))
print("A*  :", astar(graph3, "S", "G", manhattan3))


GBFS: (['S', 'A', 'G'], 3)
A*  : (['S', 'B', 'G'], 3, 4)


<b>=============================  EXERCISE  =============================</b>


<div class="box task">
<div class="boxtitle"><span class="badge b-task">✍️</span> <b>Exercise 3, A* vs GBFS (decode + run + admissibility)</b></div>

<p><b>Goal:</b> encode a weighted graph from a diagram, compare <b>GBFS</b> vs <b>A*</b>, and check whether a heuristic is <b>admissible</b>.</p>

<ol>
<li>
<b>Decode the graph</b> below into a weighted adjacency list called <code>graph_ex3</code>.
Use the same weighted format as A*, for example: <code>{"S": [("A", 2), ("B", 5)], ...}</code>.
<br>
<b>Important:</b> keep neighbour order <b>left-to-right</b> exactly as written in the diagram (this affects tie-breaking).
</li>

<li>
<b>Implement the heuristic</b> using the provided heuristic table.
Create a dictionary <code>h_ex3</code>, then define a function that returns the heuristic value for a node.
<br>
Example signature: <code>h_ex3_fn(node, goal)</code> (the goal is always <code>"G"</code> in this exercise).
</li>

<li>
<b>Run GBFS and A*</b> from <code>"S"</code> to <code>"G"</code>.
<ul>
<li>For <b>GBFS</b>, convert <code>graph_ex3</code> into an unweighted adjacency list (ignore costs) before calling <code>gbfs</code>.</li>
<li>For <b>A*</b>, call <code>astar(graph_ex3, "S", "G", h_ex3_fn)</code>.</li>
</ul>
Report: GBFS path and expanded nodes, A* path, total cost, and expanded nodes.
</li>

<li>
<b>Admissibility check (show your working)</b>.
For each node <code>S</code>, <code>A</code>, <code>B</code>, <code>C</code>, <code>D</code>:
<ul>
<li>Compute the <b>true cheapest cost</b> from that node to <code>G</code> using the edge costs in the graph.</li>
<li>Check whether <code>h(n) ≤ true_cost(n→G)</code>.</li>
</ul>
Conclude: is the heuristic <b>admissible</b>, yes/no?
</li>

<li>
<b>Modify the graph</b> so the cheapest path becomes <code>S → A → G</code>.
Change <b>only one</b> edge cost, then re-run GBFS and A* and compare the new outputs.
</li>
</ol>
</div>

<div style="padding:12px 14px;border-radius:12px;background:#ffffff;border:1px solid var(--line);
font-family:ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono','Courier New', monospace;
font-size:15px;line-height:1.5;">
<pre style="margin:0;">
Weighted directed graph (goal node G)

S -> A (2), B (1), C (4)
A -> D (2), G (7)
B -> D (5), G (9)
C -> D (1)
D -> G (2)
G -> (none)

Heuristic values h(n) to goal G (use as given):
h(S)=4, h(A)=3, h(B)=2, h(C)=3, h(D)=1, h(G)=0
</pre>
</div>

<div class="box ok">
<div class="boxtitle"><span class="badge b-ok">✅</span> What we expect to see</div>
<ul>
<li><b>GBFS</b> chooses what looks closest (low <code>h</code>), so it can ignore expensive edges.</li>
<li><b>A*</b> uses <code>f(n)=g(n)+h(n)</code>, so it accounts for cost so far and estimated remaining cost.</li>
<li><b>Admissible heuristic</b> means <code>h(n)</code> never overestimates the true cheapest cost-to-go.</li>
</ul>
</div>


In [ ]:
# TODO: Exercise 3 solution area


<details>
  <summary>✅ Show answer (Exercise 3)</summary>

  <p><b>1) One-sentence explanation</b><br>
  GBFS chooses the next node using only the heuristic <code>h(n)</code> (what looks closest), so it can ignore large edge costs, whereas A* uses <code>f(n)=g(n)+h(n)</code> and therefore accounts for the cost so far.</p>

  <p><b>2) Decode the weighted graph and heuristic table</b></p>

  <pre><code class="language-python">graph_ex3 = {
    "S": [("A", 2), ("B", 1), ("C", 4)],
    "A": [("D", 2), ("G", 7)],
    "B": [("D", 5), ("G", 9)],
    "C": [("D", 1)],
    "D": [("G", 2)],
    "G": []
}

h_ex3 = {"S": 4, "A": 3, "B": 2, "C": 3, "D": 1, "G": 0}

def h_ex3_fn(node, goal):
    return h_ex3[node]</code></pre>

  <p><b>3) Run GBFS and A*</b></p>

  <pre><code class="language-python">adj_ex3 = {k: [n for (n, _) in v] for k, v in graph_ex3.items()}

gbfs_path, gbfs_expanded = gbfs(adj_ex3, "S", "G", h_ex3_fn)
astar_path, astar_cost, astar_expanded = astar(graph_ex3, "S", "G", h_ex3_fn)

print("GBFS path:", gbfs_path, "| expanded:", gbfs_expanded)
print("A* path  :", astar_path, "| cost:", astar_cost, "| expanded:", astar_expanded)</code></pre>

  <p><b>4) Admissibility check (quick working)</b><br>
  True cheapest remaining costs to <code>G</code> are:
  <code>D=2</code>, <code>A=4</code> (via D), <code>B=7</code> (via D), <code>C=3</code>, <code>S=6</code>.
  Since <code>h(D)=1≤2</code>, <code>h(A)=3≤4</code>, <code>h(B)=2≤7</code>, <code>h(C)=3≤3</code>, <code>h(S)=4≤6</code>,
  the heuristic is <b>admissible</b>.</p>

  <p><b>5) Make the best path become <code>S → A → G</code></b><br>
  Change only one edge cost, for example reduce <code>A → G</code> from <code>7</code> to <code>3</code>,
  so <code>S → A → G</code> costs <code>2 + 3 = 5</code>, which is cheaper than any route via D (cost 6 or more).</p>

  <pre><code class="language-python">graph_ex3_mod = {
    "S": [("A", 2), ("B", 1), ("C", 4)],
    "A": [("D", 2), ("G", 3)],  # changed 7 -> 3
    "B": [("D", 5), ("G", 9)],
    "C": [("D", 1)],
    "D": [("G", 2)],
    "G": []
}

adj_ex3_mod = {k: [n for (n, _) in v] for k, v in graph_ex3_mod.items()}

print("GBFS:", gbfs(adj_ex3_mod, "S", "G", h_ex3_fn))
print("A*  :", astar(graph_ex3_mod, "S", "G", h_ex3_fn))</code></pre>

  <p><b>Expected observation</b><br>
  A* should now return <code>['S', 'A', 'G']</code> with total cost <code>5</code>. GBFS may also return that path here,
  because <code>A</code> already looks attractive under the heuristic.</p>

</details>


<b>===================================================================</b>


## 4) <b>✅ Adversarial games, a minimal Minimax example</b>

<div class="box">
  <div class="boxtitle"><span class="badge b-info">🎮</span> Core idea</div>
  <ul>
    <li>
      In adversarial (two-player, zero-sum) games, one agent tries to <b>maximise</b> a score (MAX),
      while the opponent tries to <b>minimise</b> that same score (MIN).
      The “utility” is defined from MAX’s perspective, higher is better for MAX, lower is better for MIN.
    </li>
    <li>
      <b>Minimax</b> searches a <b>game tree</b> of possible moves assuming <b>both players play optimally</b>.
      It alternates between MAX choosing the best option for itself, and MIN choosing the worst option for MAX.
    </li>
    <li>
      <b>Terminal states</b> (leaf nodes) have fixed utility values (given by the rules, or by an evaluation function).
      Internal nodes compute their value by taking either:
      <ul>
        <li><b>MAX node:</b> the <b>maximum</b> value among its children</li>
        <li><b>MIN node:</b> the <b>minimum</b> value among its children</li>
      </ul>
    </li>
    <li>
      The output of Minimax at the root tells MAX which move leads to the best guaranteed outcome, assuming MIN responds optimally.
      This is a <b>worst-case guarantee</b>, MAX chooses the move that maximises its minimum possible payoff.
    </li>
  </ul>
</div>

<div class="box">
  <div class="boxtitle"><span class="badge b-info">🧩</span> How to read a small game tree</div>
  <ul>
    <li><b>Depth</b> means how many turns ahead we look (ply). Depth 1, MAX makes one move. Depth 2, MAX then MIN responds, etc.</li>
    <li><b>Branching factor</b> is how many moves are available at each state. More branching means the tree grows quickly.</li>
    <li><b>Backing up values</b> means computing leaf utilities first, then propagating values upward using min/max rules.</li>
  </ul>
</div>

<div class="box warn">
  <div class="boxtitle"><span class="badge b-warn">⚠️</span> Keep it small</div>
  <p>
    For teaching, we use a tiny game tree with hand-written utilities rather than a full engine (like Tic-Tac-Toe).
    This keeps the focus on the <b>Minimax logic</b>, not implementation details such as move generation and game rules.
  </p>
  <p>
    In real games, trees become huge, so we typically use a <b>depth limit</b> and an <b>evaluation function</b> for non-terminal states,
    plus pruning methods (e.g., alpha–beta) to reduce the amount of search.
  </p>
</div>



<div style="padding:12px 14px;border-radius:12px;background:#ffffff;border:1px solid var(--line);
            font-family:ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono','Courier New', monospace;
            font-size:15px;line-height:1.5;">
<pre style="margin:0;">
Game tree (utilities at leaves)

          A (MAX)
        /   |    \
      B     C     D   (MIN nodes)
     / \   / \   / \
    3   5 2   9 0   7
</pre>
</div>


In [6]:
# ✅ Minimax on a small game tree
# Representation:
# - Internal nodes map to a list of children (possible moves from that position)
# - Leaves map to an integer utility (score from MAX's perspective)

children = {
    "A": ["B", "C", "D"],   # Root position A (MAX to move): MAX can choose B, C, or D
    "B": ["B1", "B2"],      # After choosing B, it is MIN's turn, MIN can choose B1 or B2
    "C": ["C1", "C2"],      # After choosing C, MIN can choose C1 or C2
    "D": ["D1", "D2"],      # After choosing D, MIN can choose D1 or D2
}

utility = {
    "B1": 3,  # Leaf utilities (terminal outcomes), higher is better for MAX
    "B2": 5,
    "C1": 2,
    "C2": 9,
    "D1": 0,
    "D2": 7,
}


def minimax(node, is_max_turn):
    """Return the minimax value of 'node' assuming optimal play from both players."""
    # Base case: if we are at a leaf, return its fixed utility value
    if node in utility:
        return utility[node]

    # Recursively compute minimax values of all children
    # Turn alternates every level: MAX -> MIN -> MAX -> ...
    vals = [minimax(ch, not is_max_turn) for ch in children[node]]

    # If it is MAX's turn at this node, choose the child with the largest value
    # If it is MIN's turn, choose the child with the smallest value (worst for MAX)
    return max(vals) if is_max_turn else min(vals)


# Evaluate the root assuming MAX plays first at A
# Interpretation: this is the guaranteed outcome MAX can force if MIN responds optimally
print("Minimax value at root A:", minimax("A", True))


Minimax value at root A: 3


<b>=============================  EXERCISE  =============================</b>


<div class="box task">
  <div class="boxtitle"><span class="badge b-task">✍️</span> Exercise 4 (Minimax decision, deeper tree)</div>
  <p>
    In this exercise, we will use a <b>3-ply</b> game tree:
    <b>A (MAX) → {B, C, D} (MIN) → {E, F, G, H, I, J} (MAX) → leaves (utilities)</b>.
  </p>

  <ol>
    <li>
      <b>By hand:</b> compute the minimax values of <b>E, F, G, H, I, J</b> (these are MAX nodes).
    </li>
    <li>
      <b>By hand:</b> using your results, compute the minimax values of <b>B, C, D</b> (these are MIN nodes).
    </li>
    <li>
      <b>By hand:</b> what is the minimax value of <b>A</b>, and which child (<b>B</b>, <b>C</b>, or <b>D</b>) should MAX choose?
    </li>
    <li>
      <b>With code:</b> run the provided <code>minimax</code> function to verify your hand calculations.
    </li>
    <li>
      <b>What-if:</b> change <b>one</b> leaf utility (your choice) so that MAX at <b>A</b> chooses a <b>different</b> child than before, then re-run minimax and report what changed.
    </li>
  </ol>

  <p><b>Game tree structure (use this exact structure):</b></p>

  <pre><code>
A (MAX)
├─ B (MIN)
│  ├─ E (MAX) → leaves: E1, E2
│  └─ F (MAX) → leaves: F1, F2
├─ C (MIN)
│  ├─ G (MAX) → leaves: G1, G2
│  └─ H (MAX) → leaves: H1, H2
└─ D (MIN)
   ├─ I (MAX) → leaves: I1, I2
   └─ J (MAX) → leaves: J1, J2
  </code></pre>

  <p><b>Leaf utilities (given):</b></p>
  <pre><code class="language-python">utility = {
    "E1": 3, "E2": 12,
    "F1": 8, "F2": 2,
    "G1": 4, "G2": 6,
    "H1": 14, "H2": 5,
    "I1": 7, "I2": 1,
    "J1": 9, "J2": 0
}</code></pre>

  <p><b>Reminder (minimax):</b> MAX takes the maximum of its children, MIN takes the minimum of its children.</p>
</div>

<div class="box ok">
  <div class="boxtitle"><span class="badge b-ok">✅</span> What we expect to see</div>
  <ul>
    <li>You can compute values bottom-up: leaves → MAX layer (E..J) → MIN layer (B..D) → root (A).</li>
    <li>Your hand-calculated choice for MAX at A matches the result produced by the minimax code.</li>
    <li>Changing a single leaf can propagate upward and change the best move at A.</li>
  </ul>
</div>


In [ ]:
# TODO: Exercise 4 solution area

# 1) By hand: compute the MAX-layer values here:
# E =
# F =
# G =
# H =
# I =
# J =

# 2) By hand: compute the MIN-layer values here:
# B =
# C =
# D =

# 3) By hand: compute A and identify the best move from A:
# A =
# Best move from A =

# 4) Verify with code:
# Run minimax("A", True) and check it matches your hand result.

# 5) What-if:
# Change ONE value in the utility dict, re-run minimax, and record the new best move.
# TODO


<details>
  <summary>✅ Answer, Exercise 4 (Minimax decision, deeper tree)</summary>

  <p><b>Game tree (children map)</b>, this must be defined for the deeper tree:</p>
  <pre><code class="language-python">children = {
  "A": ["B", "C", "D"],

  "B": ["E", "F"],
  "C": ["G", "H"],
  "D": ["I", "J"],

  "E": ["E1", "E2"],
  "F": ["F1", "F2"],
  "G": ["G1", "G2"],
  "H": ["H1", "H2"],
  "I": ["I1", "I2"],
  "J": ["J1", "J2"],
}</code></pre>

  <p><b>Leaf utilities (given)</b></p>
  <pre><code class="language-python">utility = {
  "E1": 3,  "E2": 12,
  "F1": 8,  "F2": 2,
  "G1": 4,  "G2": 6,
  "H1": 14, "H2": 5,
  "I1": 7,  "I2": 1,
  "J1": 9,  "J2": 0
}</code></pre>

  <p><b>1) MAX layer (E, F, G, H, I, J)</b></p>
  <ul>
    <li><code>E = max(E1=3, E2=12) = 12</code></li>
    <li><code>F = max(F1=8, F2=2) = 8</code></li>
    <li><code>G = max(G1=4, G2=6) = 6</code></li>
    <li><code>H = max(H1=14, H2=5) = 14</code></li>
    <li><code>I = max(I1=7, I2=1) = 7</code></li>
    <li><code>J = max(J1=9, J2=0) = 9</code></li>
  </ul>

  <p><b>2) MIN layer (B, C, D)</b></p>
  <ul>
    <li><code>B = min(E=12, F=8) = 8</code></li>
    <li><code>C = min(G=6, H=14) = 6</code></li>
    <li><code>D = min(I=7, J=9) = 7</code></li>
  </ul>

  <p><b>3) Root decision (A is MAX)</b></p>
  <p>
    <code>A = max(B=8, C=6, D=7) = 8</code><br>
    Therefore, <b>MAX should choose B</b>.
  </p>

  <p><b>4) Code verification (what you should see)</b></p>
  <pre><code class="language-python">print("B =", minimax("B", False))  # expected 8
print("C =", minimax("C", False))  # expected 6
print("D =", minimax("D", False))  # expected 7
print("A =", minimax("A", True))   # expected 8</code></pre>

  <p><b>5) What-if (change ONE leaf and observe the new best move)</b></p>
  <p>
    Example: change <code>F1</code> from 8 to 1. This reduces <code>F</code>, which reduces <code>B</code>,
    and can make <b>D</b> the best choice for MAX.
  </p>

  <pre><code class="language-python">utility["F1"] = 1  # was 8

# Now (by hand):
# F = max(F1=1, F2=2) = 2
# B = min(E=12, F=2) = 2
# C stays 6
# D stays 7
# A = max(B=2, C=6, D=7) = 7  -> MAX now chooses D

print("A after change =", minimax("A", True))</code></pre>

  <p><b>Observation:</b> MAX at A changes from choosing <b>B</b> to choosing <b>D</b>.</p>
</details>


<b>===================================================================</b>


<b>✅ Week 2 Tutorial Complete</b>

<p>
  This concludes <b>Week 2, Search and Adversarial Games</b>.
  You should now be able to represent problems as graphs, implement and compare uninformed and informed search (BFS, DFS, GBFS, A*), interpret the role of heuristic functions, and compute basic minimax decisions for simple game trees.
</p>
